In [ ]:
import torch
import data_collection
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import torch.nn as nn
import math

df = pd.read_csv('data_collection/tiktok-trending.csv')



In [ ]:
features = ['TikTok ID','User Verified','Music ID','Music Title','Music Author','Music URL','Video Duration']
x = df[features]
y = df['Play Count']

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.80, random_state=100
                                                    )
X_train = torch.tensor(x_train.values).type(torch.FloatTensor)
X_test = torch.tensor(x_test.values).type(torch.FloatTensor)
Y_train = torch.tensor(y_train.values).type(torch.FloatTensor)
Y_test = torch.tensor(y_test.values).type(torch.FloatTensor)


In [ ]:
class LogisticRegression(torch.nn.Module):
     def __init__(self):
         super(LogisticRegression, self).__init__()
         self.linear = torch.nn.Linear(x_train.shape[1],1)
     def forward(self, x):
         logits = torch.sigmoid(self.linear(x))
         return logits

seed = 12
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
epochs = 20000
learning_rate = 0.001
model = LogisticRegression()
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(),learning_rate)
losses = []
testLosses = []
# threshold = 0.5

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    Y_pred = torch.squeeze(model(X_train))    
    # print(Y_pred.shape,Y_train.shape)
    # print(Y_pred,Y_train)
    loss = criterion(Y_pred,Y_train)
    # print(loss)
    loss.backward()
    optimizer.step()
    
with torch.no_grad():
    testRight = 0
    testTot = 0
    testOut = torch.squeeze(model(X_test))
    # print(type(testOut))
    testLoss = criterion(testOut,Y_test)
    # print(Y_test.size())
    testPred =testOut
    testAcc =testOut.round().detach().numpy()
    # testPred =testOut.round().detach().numpy()
    # testPred =(testOut>threshold).detach().numpy()
    # print(type(testOut))
    testTot = Y_test.size(0)
    testRight = np.sum(testAcc == Y_test.detach().numpy())
    accuracy = 100 * testRight/testTot
    testLosses.append(testLoss.item())
    print(accuracy)